In [179]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
from sklearn.utils import check_random_state
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.metrics.pairwise import euclidean_distances, manhattan_distances, cosine_similarity
from sklearn.cluster import KMeans
from sklearn.base import BaseEstimator
from sklearn.preprocessing import StandardScaler, RobustScaler
from scipy import stats
plt.rcParams['font.family'] = 'NanumGothic'

df = pd.read_csv("../data/KmedianCluster_6_CapitalPerAsset_LiabilityPerAsset.csv", encoding = "euc-kr")
# df.drop("자본과부채총계", axis = 1, inplace = True)
idx = df.iloc[:,9:].isnull().sum().sort_values().head(11).index
idx2 = df.iloc[:,:9].columns
use_df = df[list(idx2) + list(idx)]
for col in use_df.iloc[:,9:].columns:
    print(f"{col}의 0을 가진 행의 갯수 : {use_df[use_df[col] == 0].shape[0]}")
    
# 금융부채, 유형자산, 금융자산, 비유동자산, 대손충당금
# use_df.drop(["금융부채", "유형자산", "금융자산",  "대손충당금"], axis = 1, inplace = True)

# use_df.columns[9:]

부채총계의 0을 가진 행의 갯수 : 0
유동부채의 0을 가진 행의 갯수 : 0
자본총계의 0을 가진 행의 갯수 : 0
자본총계per자산총계의 0을 가진 행의 갯수 : 0
부채총계per자산총계의 0을 가진 행의 갯수 : 0
cluster의 0을 가진 행의 갯수 : 2401
비유동부채의 0을 가진 행의 갯수 : 37
이익잉여금(결손금)의 0을 가진 행의 갯수 : 0


In [180]:
# 회사 고유값
company = df["회사명"].unique()
lst_idx = []

# 가장 최신분기에 해당하는 값 뽑기
for name in company:
    
    day = list(use_df[use_df["회사명"] == name]["결산기준일"].sort_values())[-1]
    company_index = list(use_df[(use_df["회사명"] == name) & (use_df["결산기준일"] == day)].index)[0]
    lst_idx.append(company_index)

recom_df = use_df.iloc[lst_idx, :]
recom_df.reset_index(drop = True, inplace = True)
recom_df.iloc[:,9:] = recom_df.iloc[:,9:].fillna(0)

# 코사인 값 구하기
cosine = cosine_similarity(recom_df.iloc[:,9:], recom_df.iloc[:,9:]).argsort()[:,::-1]


def get_recommend_list(df, name,  top = 30):
    cluster_num = recom_df[recom_df["회사명"] == name]["cluster"].values[0]
    new_df = recom_df[recom_df["cluster"] == cluster_num]
    new_df.reset_index(drop = True, inplace = True)
    cosine = cosine_similarity(new_df.iloc[:,9:], new_df.iloc[:,9:]).argsort()[:,::-1]
    company_index = new_df[new_df["회사명"] == name].index.values

    sim_index = cosine[company_index, :30].reshape(-1)
    sim_index = sim_index[sim_index != company_index]

    result = new_df.iloc[sim_index, :].sort_values("자본총계", ascending = False)
    return result, new_df

a, p = get_recommend_list(recom_df, "제넥신")

C:\Users\user\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [ ]:
!pip install pyecharts

In [124]:
p.head(1)

,종목코드,회사명,업종,업종명,결산기준일,보고서종류,자산총계,유동자산,비유동자산,부채총계,유동부채,비유동부채,자본총계,이익잉여금(결손금),자본총계per자산총계,부채총계per자산총계,cluster
0,[054620],APS홀딩스,649.0,기타 금융업,2020-03-31,1분기보고서,2.962370e+11,5.092961e+10,2.453074e+11,8.202884e+10,4.321294e+10,3.881591e+10,2.142082e+11,7.338660e+11,0.723097,0.276903,2


In [127]:
import pyecharts
print("pyecharts version : ", pyecharts.__version__)

pyecharts version :  1.8.1


In [142]:
from pyecharts import options as opts
from pyecharts.charts import Bar
from pyecharts.faker import Faker


c = (
    Bar()
    .add_xaxis(Faker.choose())
    .add_yaxis("商家A", Faker.values())
    .add_yaxis("商家B", Faker.values())
    .set_global_opts(
        title_opts=opts.TitleOpts(title="Bar-Y 轴 formatter"),
        yaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(formatter="{value} /月")),
    )
)


C:\Users\user\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


In [163]:
p.describe().loc["mean", :][-3:-1]

자본총계per자산총계    0.758798
부채총계per자산총계    0.241243
Name: mean, dtype: float64

array([0.81065831, 0.18934167])

In [171]:
import pyecharts.options as opts
from pyecharts.charts import Line
from pyecharts.faker import Faker

c = (
    Line()
    .add_xaxis(Faker.choose())
    .add_yaxis(
        "商家A",
        Faker.values(),
        markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="min")]),
    )
    .add_yaxis(
        "商家B",
        Faker.values(),
        markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="max")]),
    )
    .set_global_opts(title_opts=opts.TitleOpts(title="Line-MarkPoint"))
    .render("line_markpoint.html")
)

C:\Users\user\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


In [172]:
type(Faker.choose())

list

In [174]:
recom_df.iloc[:, 9:]

,부채총계,유동부채,비유동부채,자본총계,이익잉여금(결손금),자본총계per자산총계,부채총계per자산총계,cluster
0,1.712837e+10,1.449043e+10,2.637941e+09,3.519349e+10,-3.083387e+10,0.672635,0.327365,1
1,1.434722e+12,7.353061e+11,6.994162e+11,3.335254e+11,1.858849e+11,0.188619,0.811381,4
2,2.857051e+12,1.521320e+12,1.335731e+12,1.309458e+12,4.374250e+11,0.314282,0.685718,0
3,8.202884e+10,4.321294e+10,3.881591e+10,2.142082e+11,7.338660e+11,0.723097,0.276903,2
4,3.330646e+11,3.161222e+11,1.694233e+10,1.217166e+11,5.280875e+10,0.267638,0.732362,4
...,...,...,...,...,...,...,...,...
1753,2.329487e+11,1.434013e+11,8.954740e+10,6.588364e+11,4.632080e+11,0.738784,0.261216,2
1754,3.387590e+10,2.355499e+10,1.032091e+10,6.911155e+10,6.082516e+10,0.671068,0.328932,1
1755,2.542621e+10,2.350161e+10,1.924600e+09,6.260879e+10,3.423909e+10,0.711181,0.288819,2
1756,3.766872e+11,1.804243e+11,1.962629e+11,2.855110e+10,-4.825837e+10,0.070455,0.929545,4


In [181]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
a = scaler.fit_transform(recom_df.iloc[:, 9:])

In [185]:
recom_df.iloc[:, 9:] = a

C:\Users\user\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [203]:
recom_df.iloc[:, 9:].describe().loc["mean", :].index.values

array(['부채총계', '유동부채', '자본총계', '자본총계per자산총계', '부채총계per자산총계', 'cluster',
       '비유동부채', '이익잉여금(결손금)'], dtype=object)

In [194]:
df.iloc[:, 7:14] = scaler.fit_transform(df.iloc[:, 7:14])

In [199]:
df.iloc[:,7:14].describe().loc["mean", :].values

array([-4.29955929e-17, -7.88254527e-17, -6.28702947e-17,  2.78892914e-17,
        4.00995216e-17, -1.25764467e-17, -4.03936241e-17])

In [ ]:
df[df["cluster"] == 2].iloc[:, 7:14].describe().loc["mean", :].values

In [224]:
df[df["회사명"] == "삼성전자"]

,종목코드,회사명,업종,업종명,결산기준일,보고서종류,자산총계,유동자산,비유동자산,부채총계,유동부채,비유동부채,자본총계,이익잉여금(결손금),자본총계per자산총계,부채총계per자산총계,cluster
4203,[005930],삼성전자,264.0,통신 및 방송 장비 제조업,2017-12-31,사업보고서,3.017521e+14,28.245320,20.329051,12.665441,24.246648,5.872781,29.308390,31.735024,0.710820,0.289180,2
4204,[005930],삼성전자,264.0,통신 및 방송 장비 제조업,2018-12-31,사업보고서,3.393572e+14,33.597053,21.637558,13.303260,24.940123,6.600240,33.872537,35.700210,0.730066,0.269934,2
4205,[005930],삼성전자,264.0,통신 및 방송 장비 제조업,2019-03-31,1분기보고서,3.450679e+14,34.116702,22.037055,13.339776,24.311754,7.190112,34.622024,36.098689,0.733813,0.266187,2
4206,[005930],삼성전자,264.0,통신 및 방송 장비 제조업,2019-06-30,반기보고서,3.429401e+14,33.352986,22.278812,12.361824,21.385349,7.603598,35.243884,36.490953,0.751580,0.248420,2
4207,[005930],삼성전자,264.0,통신 및 방송 장비 제조업,2019-09-30,3분기보고서,3.533860e+14,35.787705,21.992666,13.059444,22.838188,7.829483,36.025375,37.036658,0.745480,0.254520,2
4208,[005930],삼성전자,264.0,통신 및 방송 장비 제조업,2019-12-31,사업보고서,3.525645e+14,34.888468,22.500101,13.021313,23.012639,7.608816,35.948283,37.452756,0.745624,0.254376,2
4209,[005930],삼성전자,264.0,통신 및 방송 장비 제조업,2020-03-31,1분기보고서,3.574575e+14,35.922413,22.439049,13.224806,23.369318,7.729774,36.429553,37.820850,0.745229,0.254771,2


In [226]:
df.iloc[11605, 7:14].values

array([-0.12945423869985767, -0.14150276412855006, -0.14508282900862207,
       -0.18061650071168767, -0.12271911427212556, -0.1156959285356583,
       -0.08638482825211789], dtype=object)

In [230]:
c = (
    Line()
    .add_xaxis(['부채총계', '유동부채', '자본총계', '자본총계per자산총계', '부채총계per자산총계', 'cluster',
       '비유동부채', '이익잉여금(결손금)'])
    .add_yaxis(
        "2군집 평균",
        [ 0.05829931,  0.00567702, -0.06619779, -0.04566687, -0.07275152,
        0.09666764,  0.11623779],
        markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="min")]),
    )
    .add_yaxis(
        "흥국피엔비",
        [-0.12945423869985767, -0.14150276412855006, -0.14508282900862207,
       -0.18061650071168767, -0.12271911427212556, -0.1156959285356583,
       -0.08638482825211789],
        markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="max")]),
    )
    .set_global_opts(title_opts=opts.TitleOpts(title="Line-MarkPoint"))
)

C:\Users\user\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


In [234]:
a = c.render_notebook()

In [ ]:
# -*- coding: utf-8 -*-
"""
adsfjadfjlkasd
"""

import pymysql
import pandas as pd
from re import sub
import numpy as np

config = {
    'host' : '127.0.0.1',
    'user' : 'scott',
    'password' : 'tiger',
    'database' : 'final',
    'port' : 3306,
    'charset' : 'utf8',
    'use_unicode' : True}



# 데이터 가져오기. final
df = pd.read_csv("C:/ITWILL/Work/Final_Project/datas_inwork/pycharm/EDAdata/항목명맞추기/!_all_preprocessing_finish_without_entity.csv", thousands=',', encoding='euc-kr')
df.info()
df.columns

# 결측치 확인
df.isnull().sum().sort_values().tail(100)

cols = list(df.columns)
len(cols)  # 198

# 총계[abstract] 삭제
abst = []
for col in cols :
    if '[abstract]' in col :
        print(col)
        abst.append(col)
'''
재무상태표 [abstract]
자산 [abstract]
부채 [abstract]
자본 [abstract]
'''

df.drop(abst, axis=1, inplace=True)
df.info()  # columns : 198 > 194



# 결측치 0으로 채우기 
df.isnull().sum()
df_fillna = df.fillna(999999)

df_fillna.iloc[0]
df_fillna.info()
df_fillna.isnull().sum().sort_values()



# bigint 컬럼 생성용 string
cols2 = list(df_fillna.columns)
len(cols2)  #  194
cols_new = cols2[9:]

from re import sub
cols_new = [ sub('[,/()]', ' ', col) for col in cols_new]
cols_eda = [ '_'.join(col.split()) for col in cols_new]

cols_add = " bigint, ".join(cols_eda) + ' bigint'

'''
cols_bigint = cols2[9:]
dtype_bigint = ['bigint']*len(cols_bigint)

data = {'colname':cols_bigint, 'dtype':dtype_bigint}
tab_add = pd.DataFrame(data=data, columns=['colname','dtype'])
tab_add.info()


for i in range(len(tab_add.colname)) :
    c = tab_add.iloc[i,0]
    d = tab_add.iloc[i,1]
    newrow = c +' '+ d
    print(newrow)
'''

# cols2도 전처리
cols2 = [ sub('[,/()]', ' ', col) for col in cols2]
cols2 = [ '_'.join(col.split()) for col in cols2]

for i, c in enumerate(cols2) :
    print(i, c)

try :
    conn = pymysql.connect(**config)
    cursor = conn.cursor()
    '''
    sql = "create table fs(종목코드 varchar(20) not null, 회사명 varchar(50) not null, 업종 int(4),\
    업종명 varchar(50), 결산기준일 varchar(20),보고서종류 varchar(30)\
    );"
    cursor.execute(sql)
    conn.commit()
    
    sql2 = f"alter table fs add({cols_add})"
    cursor.execute(sql2)
    conn.commit()
    '''
    sql3 = "desc fs"
    cursor.execute(sql3)

    dataset = cursor.fetchall()
    for row in dataset :
        print(row)
        
        
        
    
    """
    for i in range(len(df_fillna['종목코드'])):
        for idx, col in enumerate(cols2) :
        '''
        col = df_fillna.iloc[i][idx]
        회사명 = df_fillna.iloc[i][1]
        업종 = df_fillna.iloc[i][2]
        업종명 = df_fillna.iloc[i][3]
        결산기준일 = df_fillna.iloc[i][4]
        보고서종류 = df_fillna.iloc[i][5]
        유동자산 = df_fillna.iloc[i][6]
        현금및현금성자산 = df_fillna.iloc[i][7]
        비유동자산 = df_fillna.iloc[i][8]
        유형자산 = df_fillna.iloc[i][9]
        자산총계 = df_fillna.iloc[i][10]
        유동부채 = df_fillna.iloc[i][11]
        비유동부채 = df_fillna.iloc[i][12]
        부채총계 = df_fillna.iloc[i][13]
        자본총계 = df_fillna.iloc[i][14]
        자본과부채총계 = df_fillna.iloc[i][15]
        print(종목코드)
        print(회사명)
        print(유동자산)
        '''
        
        sql_insert = f"insert into wo_entity_simple values('{종목코드}','{회사명}',{업종},'{업종명}','{결산기준일}','{보고서종류}',{유동자산},{현금및현금성자산},{비유동자산},{유형자산},{자산총계},{유동부채},{비유동부채},{부채총계},{자본총계},{자본과부채총계})"
        cursor.execute(sql_insert)
        conn.commit()
        print("insert 완료 행 :", i)
        
    print('data insert executed')

    sql = "select * from wo_entity_simple"
    cursor.execute(sql)
    dataset_insert = cursor.fetchall()

    for row in dataset_insert :
        print(row)
    print("show results")
    """
    
    
    
    
    
except Exception as e :
    print('db error :', e)
finally :
    cursor.close()
    conn.close()